<a href="https://colab.research.google.com/github/Deepjyot-ML-workspace/Calculating-Value-at-Risk-VaR-of-portfolio/blob/main/Calculating_Value_at_Risk_(VaR)_of_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



<p>Suppose you're given a portfolio of equities and asked to calculate the 'value at risk' (VaR) via the <a href="https://www.investopedia.com/ask/answers/041715/what-variancecovariance-matrix-or-parametric-method-value-risk-var.asp" target="_blank">variance-covariance method</a>.</p>
<p>The VaR is a statistical risk management technique measuring the maximum loss that an investment portfolio is likely to face within a specified time frame with a certain degree of confidence. The VaR is a commonly calculated metric used within a suite of financial metrics and models to help aid in investment decisions.</p>
<p>In order to calculate the VaR of your portfolio, you can follow the steps below:</p>
<ol>
  <li>Calculate periodic returns of the stocks in your portfolio</li>
  <li>Create a covariance matrix based on (1)</li>
  <li>Calculate the portfolio mean and standard deviation (weighted based on investment levels of each stock in portfolio)</li>
  <li>Calculate the inverse of the normal cumulative distribution with a specified probability, standard deviation, and mean</li>
   <li>Estimate the value at risk for the portfolio by subtracting the initial investment from the calculation in step 4</li>
  </ol>
  <p>To help get you started, you can reference this <a href="https://colab.research.google.com/drive/1dPrUZocrhG1dWyZP33jGaXiKg1oFSdpi">Google Colab</a> notebook with the historical returns for a portfolio of the following equities:
  <pre>['AAPL','FB', 'C', 'DIS']</pre>
  </p>
  <p>Full solution will be provided in python to premium users.</p>



In [ ]:
import pandas as pd
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
import numpy as np
import datetime as dt

#Create our portfolio of equities
tickers = ['AAPL','FB', 'C', 'DIS']

#Set the investment weights (I arbitrarily picked for example)
weights = np.array([.25, .3, .15, .3])

#Set an initial investment level
initial_investment = 1000000

#Download closing prices
data = pdr.get_data_yahoo(tickers, start="2018-01-01", end=dt.date.today())['Close']

#From the closing prices, calculate periodic returns
returns = data.pct_change()

In [ ]:
returns.head()

Symbols,AAPL,C,DIS,FB
Date,,,,
2018-01-02,NaN,NaN,NaN,NaN
2018-01-03,-0.000174,0.003093,0.004293,0.017914
2018-01-04,0.004645,0.012334,-0.000445,-0.001841
2018-01-05,0.011385,-0.001324,-0.005435,0.013671
2018-01-08,-0.003714,-0.011670,-0.014334,0.007653




---




**Solution:**


In [ ]:
from scipy.stats import norm

#Generate Var-Cov matrix
cov_matrix = returns.cov()

#Calculate mean returns for each stock
avg_rets = returns.mean()

#Calculate mean returns for portfolio overall,
#using mean, using dot product formula to
#normalize against investment weights
port_mean = avg_rets.dot(weights)

#Calculate portfolio standard deviation
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))

#Calculate mean of given investment
mean_investment = (1+port_mean) * initial_investment

#Calculate standard deviation of given investmnet
stdev_investment = initial_investment * port_stdev

#Select our confidence interval (I'll choose 95% here)
conf_level1 = 0.05

#Using SciPy ppf method to generate values for the
#inverse cumulative distribution function to a normal distribution
#Plugging in the mean, standard deviation of our portfolio
#as calculated above
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html
cutoff1 = norm.ppf(conf_level1, mean_investment, stdev_investment)

#Finally, we can calculate the VaR at our confidence interval
var_1d1 = initial_investment - cutoff1
var_1d1

22347.7792230231

<p>Here we are saying with 95% confidence that the loss of our portfolio will not exceed ~$22.3k over a one day period.</p>
<p>What if we wanted to calculate this over a larger window of time? Below we can easily do that by just taking our 1 day VaR and multiplying it by the <a href="https://www.investopedia.com/articles/04/101304.asp" target="_blank">square root of the time period</a>.</p>

In [ ]:
#Calculate n Day VaR
num_days = 30
for x in range(1, num_days+1):
    print(str(x) + " day VaR @ 95% confidence: " + str(np.round(var_1d1 * np.sqrt(x),2)))

1 day VaR @ 95% confidence: 22347.78
2 day VaR @ 95% confidence: 31604.53
3 day VaR @ 95% confidence: 38707.49
4 day VaR @ 95% confidence: 44695.56
5 day VaR @ 95% confidence: 49971.15
6 day VaR @ 95% confidence: 54740.66
7 day VaR @ 95% confidence: 59126.67
8 day VaR @ 95% confidence: 63209.06
9 day VaR @ 95% confidence: 67043.34
10 day VaR @ 95% confidence: 70669.88
11 day VaR @ 95% confidence: 74119.2
12 day VaR @ 95% confidence: 77414.98
13 day VaR @ 95% confidence: 80576.06
14 day VaR @ 95% confidence: 83617.73
15 day VaR @ 95% confidence: 86552.58
16 day VaR @ 95% confidence: 89391.12
17 day VaR @ 95% confidence: 92142.25
18 day VaR @ 95% confidence: 94813.6
19 day VaR @ 95% confidence: 97411.71
20 day VaR @ 95% confidence: 99942.31
21 day VaR @ 95% confidence: 102410.39
22 day VaR @ 95% confidence: 104820.38
23 day VaR @ 95% confidence: 107176.18
24 day VaR @ 95% confidence: 109481.31
25 day VaR @ 95% confidence: 111738.9
26 day VaR @ 95% confidence: 113951.76
27 day VaR @ 95% c